

---


**Notebook pour le test de modèle pour le PTRANS**

☕*Created by :*

*   Mattéo Boursault
*   Ibrahim Braham
*   Adam Creusevault

Dernière modification : *18/05/2022*

---



#### Import + Connexion au Drive + Modèle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install albumentations==0.4.6
!pip install torchinfo

import torch
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2
import copy
import torch.nn as nn

In [ ]:
# modele UNet :

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Transposed convolution then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels , out_channels, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)


    def forward_padding(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = nn.functionnal.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

    def forward(self, target_tensor, contracting_tensor):
        target_tensor = self.up(target_tensor)
        target_height = target_tensor.size()[2]
        target_width = target_tensor.size()[3]
        crop = transforms.CenterCrop((target_height, target_width))

        contracting_tensor = crop(contracting_tensor)
        new_tensor = torch.cat((target_tensor, contracting_tensor), 1)

        return self.conv(new_tensor)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        self.up1 = Up(1024, 512)
        self.up2 = Up(512, 256)
        self.up3 = Up(256, 128)
        self.up4 = Up(128, 64)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

#### Configuration

In [ ]:
'''
- Complétez les chemins
- Executez toutes les cellules du Notebook
- Have fun
'''

IMG_PATH = "drive/MyDrive/soutenance/radio_1.jpg"
MODEL_PATH = 'drive/MyDrive/soutenance/model.pth'
RESIZING_SIZE = 1024

#### Test du modèle

In [ ]:
# For testing new image : complete path

# Load the model
model = UNet(1, 4)
model_to_test = torch.load(MODEL_PATH)
model.load_state_dict(model_to_test["model_state_dict"])
model.eval()
model.to("cuda")

radio = np.array(Image.open(IMG_PATH))

# Resizing
validation_transform = A.Compose([
    A.LongestMaxSize(RESIZING_SIZE),
    A.PadIfNeeded(min_height=RESIZING_SIZE, min_width=RESIZING_SIZE, border_mode=cv2.BORDER_CONSTANT),
])
radio = validation_transform(image=radio)['image']

# Grayscale
grayscale_transform = transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()])
radio = Image.fromarray(radio)
radio = grayscale_transform(radio)

output = model(radio.unsqueeze(1).to("cuda"))
output = output[0].cpu().data.numpy()
index = output.argmax(axis=0)

figure, ax = plt.subplots(ncols=2, figsize=(10, 40))
ax[0].imshow(radio[0])
ax[1].imshow(index*100)

ax[0].set_title("Image: ", fontsize=20)
ax[1].set_title("predicted mask: ", fontsize=20)

ax[0].set_axis_off()
ax[1].set_axis_off()

plt.tight_layout()
plt.show()
